## from_full_data_to_full_data_all_vars.ipynb

In this notebook, I extract all remaining variables, for an analysis with covariates. It should be modeled after `find_remaining_variables.ipynb`, with the exception that we have to add the death date at the beginning. Then, the script can be modeled more or less exactly after the aforementioned notebook. So, the approach is:

   - Add the sterfdatum
   - Redo the entire script as in `find_remaining_variables.ipynb` 
   

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import re
from itertools import compress

from matplotlib import style
from matplotlib import pyplot as plt
import seaborn as sns

from rdrobust import rdrobust,rdbwselect,rdplot

/home/bas/anaconda3/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.


In [17]:
data = pd.read_csv("../Data/analysis/full_sample_analysis_novars.csv", dtype={'b1-nummer':str}).iloc[:, 1:]
data

,Naam,name_in_all_elections,name_in_elected_people,Aanbevolen door,Aantal stemmen,Procentueel,District,Verkiezingdatum,Type,Omvang electoraat,...,deflated_wealth,verk_2_gewonnen,verk_3_gewonnen,verk_4_gewonnen,verk_5_gewonnen,verk_6_gewonnen,verk_7_gewonnen,verk_8_gewonnen,verk_9_gewonnen,verk_10_gewonnen
0,S.A. de Moraaz,S.A. de Moraaz,S.A. de Moraaz,NaN,503.0,52.84%,Alkmaar,1848-11-30,algemeen,1107,...,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,G. van Leeuwen,NaN,NaN,NaN,438.0,46.01%,Alkmaar,1848-11-30,algemeen,1107,...,12355.953890,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,mr. H.J. Smit,mr. H.J. Smit,H.J. Smit,NaN,1566.0,79.86%,Alkmaar,1850-08-27,algemeen,2833,...,306967.902828,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,S.A. de Moraaz,S.A. de Moraaz,S.A. de Moraaz,NaN,1275.0,65.02%,Alkmaar,1850-08-27,algemeen,2833,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jhr.mr. C. van Foreest,jhr.mr. C. van Foreest,C. van Foreest,NaN,685.0,34.93%,Alkmaar,1850-08-27,algemeen,2833,...,NaN,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8454,B. Luteraan,NaN,NaN,SDP,32.0,0.41%,Zwolle,1913-06-17,algemeen,8889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8455,F.M. Knobel,F.M. Knobel,F.M. Knobel,VL(Lib/VD/SDAP),4249.0,50.15%,Zwolle,1913-06-25,herstemming,8889,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8456,A. baron van Dedem,A. baron van Dedem,A. baron van Dedem,CHU(Ka/AR),4223.0,49.85%,Zwolle,1913-06-25,herstemming,8889,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8457,F.M. Knobel,F.M. Knobel,F.M. Knobel,VL,3236.0,86.22%,Zwolle,1917-06-15,algemeen,9645,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
## Add the sterfdatum
sterfdatum_politici = pd.read_excel("../Data/politician_data/tk_1815tot1950uu.xlsx", 
              dtype={'b1-nummer':str},
              sheet_name=1)
sterfdatum_politici = sterfdatum_politici[sterfdatum_politici['rubriek'] == 3020][['b1-nummer', 'datum']]

data = pd.merge(data, sterfdatum_politici, how='left',
         left_on='b1-nummer',
         right_on='b1-nummer').rename(columns={'datum':'sterfdatum'})

In [30]:
nonpols = pd.read_csv('../Data/analysis/unmatched_sample_analysis.csv').iloc[:,1:]
nonpols = nonpols[nonpols['b1-nummer'].isna()][['Naam', 'Sterfdatum']].drop_duplicates()
nonpols = nonpols[~nonpols['Sterfdatum'].isna()]

In [38]:
data = pd.merge(data, nonpols,
        how='left',
        left_on='Naam',
         right_on='Naam').rename(columns={'Sterfdatum':'Dod'})

data['Sterfdatum'] = np.where(data['sterfdatum'].isna(), data['Dod'], data['sterfdatum'])

data = data.drop(columns=['sterfdatum', 'Dod'])

In [40]:
# from here, proceed as in find_remaining_variables